In [2]:
pip install python-docx
pip install beautifulsoup4



   ---------------------------------------- 0.0/239.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/239.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/239.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/239.6 kB ? eta -:--:--
   ----- --------------------------------- 30.7/239.6 kB 145.2 kB/s eta 0:00:02
   ----- --------------------------------- 30.7/239.6 kB 145.2 kB/s eta 0:00:02
   ---------- ---------------------------- 61.4/239.6 kB 204.8 kB/s eta 0:00:01
   --------------- ----------------------- 92.2/239.6 kB 261.7 kB/s eta 0:00:01
   ---------------------- --------------- 143.4/239.6 kB 387.0 kB/s eta 0:00:01
   -------------------------------------  235.5/239.6 kB 554.9 kB/s eta 0:00:01
   -------------------------------------  235.5/239.6 kB 554.9 kB/s eta 0:00:01
   -------------------------------------- 239.6/239.6 kB 473.9 kB/s eta 0:00:00


In [22]:
import requests
from docx import Document
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image

def html_to_docx(input_path, output_file):
    if urlparse(input_path).scheme in ('http', 'https'):
        # Fetch the HTML content from the URL
        response = requests.get(input_path)
        html_content = response.text
    else:
        # Read the HTML content from the file
        with open(input_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Create a new Word document
    doc = Document()

    # Find all elements in the HTML and add them to the Word document
    for element in soup.find_all():
        if element.name == 'p':
            add_paragraph(doc, element)
        elif element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            add_heading(doc, element)
        elif element.name == 'img':
            add_image(doc, element)

    # Save the Word document
    doc.save(output_file)

def add_paragraph(doc, element):
    paragraph = doc.add_paragraph()
    for elem in element.contents:
        if elem.name == 'span':
            add_run(paragraph, elem)
        else:
            paragraph.add_run(elem.get_text())

def add_heading(doc, element):
    level = int(element.name[1])
    heading = doc.add_heading(level=level)
    for elem in element.contents:
        if elem.name == 'span':
            add_run(heading, elem)
        else:
            heading.add_run(elem.get_text())

def add_run(paragraph, element):
    run = paragraph.add_run()
    style = element.get('style')
    if style:
        if 'color' in style:
            color = style.split('color:', 1)[1].split(';')[0]
            run.font.color.rgb = hex_to_rgb(color)
        if 'font-weight:bold' in style:
            run.bold = True
        if 'font-style:italic' in style:
            run.italic = True
        if 'text-decoration:underline' in style:
            run.underline = True
    run.text = element.get_text()

def add_image(doc, element):
    if 'src' in element.attrs:
        image_url = element['src']
        response = requests.get(image_url)
        image_data = BytesIO(response.content)
        img = Image.open(image_data)
        width, height = img.size
        if width > 500:
            new_width = 500
            new_height = int((new_width / width) * height)
            img = img.resize((new_width, new_height))
        doc.add_picture(image_data)

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

# Example usage
#html_to_docx('input.html', 'output.docx')  # For local file
html_to_docx('https://example.com/page.html', 'output.docx')  # For web URL

In [25]:
import requests
from docx import Document
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image
import base64

def html_to_docx(input_path, output_file):
    if urlparse(input_path).scheme in ('http', 'https'):
        # Fetch the HTML content from the URL
        response = requests.get(input_path)
        html_content = response.text
    else:
        # Read the HTML content from the file
        with open(input_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Create a new Word document
    doc = Document()

    # Find all elements in the HTML and add them to the Word document
    for element in soup.find_all():
        if element.name == 'p':
            add_paragraph(doc, element)
        elif element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            add_heading(doc, element)
        elif element.name == 'img':
            add_image(doc, element)

    # Save the Word document
    doc.save(output_file)

def add_paragraph(doc, element):
    paragraph = doc.add_paragraph()
    for elem in element.contents:
        if elem.name == 'span':
            add_run(paragraph, elem)
        else:
            paragraph.add_run(elem.get_text())

def add_heading(doc, element):
    level = int(element.name[1])
    heading = doc.add_heading(level=level)
    for elem in element.contents:
        if elem.name == 'span':
            add_run(heading, elem)
        else:
            heading.add_run(elem.get_text())

def add_run(paragraph, element):
    run = paragraph.add_run()
    style = element.get('style')
    if style:
        if 'color' in style:
            color = style.split('color:', 1)[1].split(';')[0]
            run.font.color.rgb = hex_to_rgb(color)
        if 'font-weight:bold' in style:
            run.bold = True
        if 'font-style:italic' in style:
            run.italic = True
        if 'text-decoration:underline' in style:
            run.underline = True
    run.text = element.get_text()

def add_image(doc, element):
    if 'src' in element.attrs:
        image_source = element['src']
        if image_source.startswith('data:image'):
            # Image source is a base64-encoded image
            image_data = BytesIO(base64.b64decode(image_source.split(',')[1]))
        elif urlparse(image_source).scheme in ('http', 'https'):
            # Image source is a URL
            response = requests.get(image_source)
            image_data = BytesIO(response.content)
        else:
            # Image source is a local file path
            with open(image_source, 'rb') as f:
                image_data = BytesIO(f.read())
        
        img = Image.open(image_data)
        width, height = img.size
        if width > 500:
            new_width = 500
            new_height = int((new_width / width) * height)
            img = img.resize((new_width, new_height))
        doc.add_picture(image_data)

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

# Example usage
#html_to_docx('input.html', 'output.docx')  # For local file
html_to_docx('https://www.jumia.co.ke/', 'output.docx')  # For web URL
